<a href="https://colab.research.google.com/github/aozdl/algorithm-optimization-project-machine-learning/blob/main/sql_prqctice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EXERCICE 1**

# **All installations**

In [91]:
%pip install pysqlite3
%pip install pandas
%pip install numpy

# **All imports**

In [103]:
import sqlite3 as sql
import pandas as pd
import numpy as np


# **Connection to the Database**

In [107]:
con = sqlite3.connect('employees.db')

In [109]:
#con.execute('DROP TABLE employees')

In [110]:
con.execute('''CREATE TABLE employees (
  employee_id INTEGER PRIMARY KEY,
  name TEXT,
  department_id INTEGER,
  salary INTEGER
)''')

In [111]:
#con.execute('SHOW tables');

In [112]:
con.execute('''CREATE TABLE departments (
  department_id INTEGER PRIMARY KEY,
  name TEXT
)''')

In [23]:
con.execute('INSERT INTO employees (name, department_id, salary) VALUES ("Ayse", "1", "50000")')

In [24]:
con.execute('INSERT INTO employees (name, department_id, salary) VALUES ("Fernanda", "2", "70000")')

In [25]:
con.execute('INSERT INTO employees (name, department_id, salary) VALUES ("Julien", "3", "90000")')

In [26]:
con.execute('INSERT INTO employees (name, department_id, salary) VALUES ("David", "4", "90000")')

In [27]:
con.execute('INSERT INTO departments (name) VALUES ("Administration")')

In [28]:
con.execute('INSERT INTO departments (name) VALUES ("Francescos Department")')

In [41]:
out = con.execute("SELECT * FROM employees")
out.fetchall()

[(1, 'Ayse', 1, 50000), (3, 'Julien', 3, 90000), (4, 'David', 4, 90000)]

In [32]:
con.execute('DELETE FROM employees WHERE employee_id = 2')

In [42]:
out = con.execute("SELECT * FROM departments")
out.fetchall()

[(1, 'Administration'),
 (2, 'Francescos Department'),
 (3, 'Data Science Department')]

In [44]:
result = con.execute('SELECT employees.name, departments.name FROM employees INNER JOIN departments ON employees.department_id = departments.department_id')
for row in result:
  print(row)


('Ayse', 'Administration')
('Julien', 'Data Science Department')


In [45]:
con.execute('INSERT INTO departments (name) VALUES ("Data Science Department")')

In [46]:
result = con.execute('SELECT * FROM employees INNER JOIN departments ON employees.department_id = departments.department_id')
for row in result:
  print(row)

(1, 'Ayse', 1, 50000, 1, 'Administration')
(3, 'Julien', 3, 90000, 3, 'Data Science Department')
(4, 'David', 4, 90000, 4, 'Data Science Department')


In [47]:
employees_df = pd.read_sql_query("SELECT * FROM employees", con)
employees_df

,employee_id,name,department_id,salary
0,1,Ayse,1,50000
1,3,Julien,3,90000
2,4,David,4,90000


In [48]:
departments_df = pd.read_sql_query("SELECT * FROM departments", con)
departments_df

,department_id,name
0,1,Administration
1,2,Francescos Department
2,3,Data Science Department
3,4,Data Science Department


In [54]:
result_df = pd.read_sql_query('SELECT employees.name, departments.name FROM employees INNER JOIN departments ON employees.department_id = departments.department_id', con)
result_df

,name,name
0,Ayse,Administration
1,Julien,Data Science Department
2,David,Data Science Department


In [113]:
con.execute('INSERT INTO employees (name, department_id, salary) VALUES ("Jason", "5", "40000")')

In [116]:
result_df = pd.merge(employees_df, departments_df, on='department_id')
result_df[['name_x', 'name_y']]

,name_x,name_y
0,Ayse,Administration
1,Julien,Data Science Department
2,David,Data Science Department


# **EXERCICE 2**

In [55]:
con = sqlite3.connect('grades.db')

In [76]:
con.execute('DROP TABLE grades')

In [77]:
con.execute('''CREATE TABLE grades (
  student_name TEXT,
  class TEXT,
  grade INTEGER
)''')

In [78]:
grades_l = [('Julian', 'DS', 8), ('Jason', 'Math', '10'), ('Jason', 'DS', 9), ('David', 'DS', 9), ('Fernanda', 'DS', 7), ('Fernanda', 'Math', 8), ('Ayse', 'Maths', 9), ('Ayse', 'DS', 7)]

In [79]:
con.executemany('INSERT INTO grades VALUES (?,?,?)', grades_l)

In [80]:
out = con.execute("SELECT * FROM grades")
out.fetchall()

[('Julian', 'DS', 8),
 ('Jason', 'Math', 10),
 ('Jason', 'DS', 9),
 ('David', 'DS', 9),
 ('Fernanda', 'DS', 7),
 ('Fernanda', 'Math', 8),
 ('Ayse', 'Maths', 9),
 ('Ayse', 'DS', 7)]

In [81]:
result = con.execute('SELECT class, AVG(grade) FROM grades GROUP BY class')
for row in result:
  print(row)

('DS', 8.0)
('Math', 9.0)
('Maths', 9.0)


In [82]:
result = con.execute('SELECT student_name, AVG(grade) FROM grades GROUP BY student_name')
for row in result:
  print(row)

('Ayse', 8.0)
('David', 9.0)
('Fernanda', 7.5)
('Jason', 9.5)
('Julian', 8.0)


In [83]:
grades_df = pd.read_sql_query("SELECT * FROM grades", con)
grades_df

,student_name,class,grade
0,Julian,DS,8
1,Jason,Math,10
2,Jason,DS,9
3,David,DS,9
4,Fernanda,DS,7
5,Fernanda,Math,8
6,Ayse,Maths,9
7,Ayse,DS,7


In [85]:
results_df = grades_df.groupby('class')['grade'].mean()
results_df

class
DS       8.0
Math     9.0
Maths    9.0
Name: grade, dtype: float64

In [87]:
results_df = grades_df.groupby('student_name')['grade'].mean()
results_df

student_name
Ayse        8.0
David       9.0
Fernanda    7.5
Jason       9.5
Julian      8.0
Name: grade, dtype: float64